A simplified workthrough of Mask RCNN training. Currently contains no evaluation.

In [5]:
import os
import sys
import itertools
from statistics import mean
from time import time
from tqdm import tqdm
sys.path.append('..')

Set XLA to fusible. This will fuse smaller operations, but leave larger operations as is. So far, this seems to give the biggest performance gains.

In [2]:
os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=fusible"

Disable all TF 2 behavior. Set TF to use GPU 0.

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

import horovod.tensorflow as hvd
hvd.init()

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[hvd.rank()], 'GPU')
devices = tf.config.list_logical_devices('GPU')

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
from mask_rcnn.hyperparameters import dataset_params
from mask_rcnn.hyperparameters import mask_rcnn_params
from mask_rcnn import dataset_utils
import load_weights, model

Train data can be found at s3://[redacted]/data/coco/tfr_anchor/

In [12]:
train_file_pattern = '/home/ubuntu/data/coco/tf_record/train*'
batch_size = 6
data_params = dataset_params.get_data_params()
params = mask_rcnn_params.default_config().values()
data_params['batch_size'] = batch_size
params['finetune_bn'] = False
params['train_batch_size'] = batch_size
params['l2_weight_decay'] = 1e-4
params['init_learning_rate'] = 1e-4 * batch_size
params['warmup_learning_rate'] = 1e-3 * batch_size
params['warmup_steps'] = 500
params['learning_rate_steps'] = [30000,40000]
params['learning_rate_levels'] = [1e-4 * batch_size, 1e-5 * batch_size]
params['momentum'] = 0.9
params['use_batched_nms'] = True

Fast data loader is a modified data loader that handles precomputed box matches.

In [14]:
train_input_fn = dataset_utils.FastDataLoader(train_file_pattern, data_params)
train_tdf = train_input_fn(data_params)
tdf_iter = train_tdf.make_initializable_iterator()
features, labels = tdf_iter.get_next()

[MaskRCNN] INFO    : Using Dataset Sharding with Horovod
Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_initializable_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.


In [15]:
train_op, total_loss = model.model(features, params, labels)

Load pretrained resnet weights. Make sure to first run the ./download_and_process_pretrained_weights.sh script.

In [17]:
var_list = load_weights.build_assigment_map('resnet50/')
checkpoint_file = tf.train.latest_checkpoint('../resnet/resnet-nhwc-2018-02-07/')
_init_op, _init_feed_dict = load_weights.assign_from_checkpoint(checkpoint_file, var_list)

Training loop. To run multi-gpu with horovod, use the run.sh script.

In [ ]:
var_initializer = tf.global_variables_initializer()
loss_history = []
with tf.Session() as sess:
    sess.run(_init_op, _init_feed_dict)
    sess.run(tdf_iter.initializer)
    sess.run(var_initializer)
    progressbar = tqdm(range(10000))
    for i in progressbar:
        op, loss = sess.run((train_op, total_loss))
        loss_history.append(loss)
        smoothed_loss = mean(loss_history[-50:])
        progressbar.set_description("Loss: {0:.4f}".format(smoothed_loss))

[GPU 00] Restoring pretrained weights (265 Tensors) from: ../resnet/resnet-nhwc-2018-02-07/model.ckpt-112603


Loss: 2.5485:   3%|▎         | 262/10000 [02:28<47:48,  3.39it/s]  